In [ ]:
! pip install requests -q
! pip install html5lib -q
! pip install bs4 -q
! pip install tiktoken -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.1/201.1 kB 4.2 MB/s eta 0:00:00


In [ ]:
import requests
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
import pandas as pd
import tiktoken
from google.colab import files

In [ ]:
base_url = "https://www.memorial.com.tr"
def set_url(base_url, param):
  return f"{base_url}/{param}"

In [ ]:
def get_divs(url):
  res = requests.get(url)
  soup = BeautifulSoup(res.content, "html.parser")
  htmlData = soup.find_all("div", {"class": "col-3 mini-image-box"})
  return htmlData

In [ ]:
def get_names(htmlData):
  names = []
  for html in htmlData:
    name = html.find("h3").text
    name = name.replace("\n", "")
    name = name.replace("\r", "")
    name = name.strip()
    names.append(name)
  return names

In [ ]:
def get_raw_texts(params):
  raw_texts = []
  for p in params:
    url = set_url(base_url, p)
    res = requests.get(url)
    soup = BeautifulSoup(res.content, "html.parser")
    div = soup.find_all("div", {"class": "article"}) # getting the main content
    texts = div[0].find_all(["h2", "h3", "p"])
    raw_texts.append(texts)
  return raw_texts

In [ ]:
def get_raw_text(record):
  name = record["name"]
  param = record["param"]
  url = set_url(base_url, param)
  res = requests.get(url)
  soup = BeautifulSoup(res.content, "html.parser")
  div = soup.find_all("div", {"class": "article"}) # getting the main content
  texts = div[0].find_all(["h2", "h3", "p"])
  rec = {
      name: texts
  }
  raw_texts.append(rec)
  print(f"successfuly extracted raw content of {param}")

def get_raw_text_pool(max_workers, param_records):
  with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = [
        executor.submit(get_raw_text, record) for record in param_records
    ]
    for future in as_completed(futures):
      try:
        future.result()
      except Exception as e:
        print(f"Thread Error: {e}")

In [ ]:
def get_texts(raw_data, k):
  # k : decides which part of the raw content will be deleted from the end
  # useful for deleting non-relevant data present at the end
  texts = []
  for d in raw_data:
    data = d.copy()
    del data[-k:] # deletes the last k elements
    text = ""
    for element in data:
      text += element.text
    texts.append(" " + text)
  return texts

In [ ]:
def num_tokens_from_string(string: str, encoder_name: str) -> int:
    # encoding = tiktoken.encoding_for_model(model_name)
    encoding = tiktoken.get_encoding(encoder_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [ ]:
def get_num_tokens(encoder_name, texts):
  num_tokens = []
  for text in texts:
    num_token = num_tokens_from_string(text, encoder_name)
    num_tokens.append(num_token)
  return num_tokens

# Tanı ve Testler (Diagnoses and Tests)

In [ ]:
param = "tani-ve-testler"
url = set_url(base_url, param)
url

'https://www.memorial.com.tr/tani-ve-testler'

In [ ]:
htmlData = get_divs(url)

In [ ]:
names = get_names(htmlData)
names[:3], len(names)

(['Tiroid Biyopsisi',
  '3 Boyutlu Ekokardiyografi',
  'ACTH (Adrenokortikotropik Hormon) Testi'],
 141)

In [ ]:
name_params = []
for html in htmlData:
  url = [a['href'] for a in html.select('a[href^="/tani-ve-testler"]')]
  name_params.append(url[0])

In [ ]:
len(name_params)

141

In [ ]:
raw_texts = get_raw_texts(name_params)    # takes about 34sec

In [ ]:
texts = get_texts(raw_texts, k=4)   # k was checked beforehand

In [ ]:
encoder = "cl100k_base"
num_tokens = get_num_tokens(encoder, texts)

In [ ]:
data = {
    "category": "Tanı ve Testler",
    "topic": names,
    "text": texts,
    "num_tokens": num_tokens
}

In [ ]:
df = pd.DataFrame(data=data, index=None)

In [ ]:
df

,category,topic,text,num_tokens
0,Tanı ve Testler,Tiroid Biyopsisi,Tiroid bezini oluşturan hücrelerin düzensiz ç...,3501
1,Tanı ve Testler,3 Boyutlu Ekokardiyografi,Bilgisayar ve transdüser sistemlerindeki tekn...,2117
2,Tanı ve Testler,ACTH (Adrenokortikotropik Hormon) Testi,"ACTH (adrenokortikotropik hormon), hipofiz be...",2363
3,Tanı ve Testler,Akciğer Grafisi,"Akciğer grafisi; kalp, akciğerler, kan damarl...",1753
4,Tanı ve Testler,Albümin Testi,"Albümin testi, karaciğer ve böbrek fonksiyonl...",3426
...,...,...,...,...
136,Tanı ve Testler,Uyku Apnesi Testi,Kaliteli ve yeterli olmayan uyku süresi kişin...,3427
137,Tanı ve Testler,Üre (BUN Testi),"Üre, karaciğerin proteinin sindirimi sürecind...",3579
138,Tanı ve Testler,Üre Nefes Testi,Dünyanın en yaygın enfeksiyonlarından biri ol...,1142
139,Tanı ve Testler,Ürodinami Testi,Erkeklerde ve kadınlarda hemen hemen her yaşt...,2850


In [ ]:
df.to_pickle("tani-ve-testler.pkl")

# Hastalıklar (Diseases)

In [ ]:
param = "hastaliklar"
url = set_url(base_url, param)
url

'https://www.memorial.com.tr/hastaliklar'

In [ ]:
htmlData = get_divs(url)

In [ ]:
names = get_names(htmlData)
names[:3], len(names)

(['20’lik Diş', 'Addison Hastalığı', 'Adenit'], 569)

In [ ]:
name_params = []
for html in htmlData:
  url = [a['href'] for a in html.select('a[href^="/hastaliklar"]')]
  name_params.append(url[0])
len(name_params)

569

In [ ]:
# creating a list of dicts to keep track of the data while running ThreadPoolExecutor
param_records = []
for i in range(len(names)):
  rec = {
      "name": names[i],
      "param": name_params[i]
  }
  param_records.append(rec)

In [ ]:
raw_texts = []
get_raw_text_pool(10, param_records)      # takes about 1min to complete

In [ ]:
len(raw_texts)    # checking if all the records are fetched

569

In [ ]:
# getting raw text and names in a list format
names = []
raws = []
for rec in raw_texts:
  names.append(list(rec.keys())[0])
  raws.append(list(rec.values())[0])

In [ ]:
texts = get_texts(raws, k=7)    # k was checked beforehand

In [ ]:
encoder = "cl100k_base"
num_tokens = get_num_tokens(encoder, texts)

In [ ]:
data = {
    "category": "Hastalıklar",
    "topic": names,
    "text": texts,
    "num_tokens": num_tokens
}

In [ ]:
df = pd.DataFrame(data=data, index=None)

In [ ]:
df

,category,topic,text,num_tokens
0,Tanı ve Testler,Tiroid Biyopsisi,Tiroid bezini oluşturan hücrelerin düzensiz ç...,3501
1,Tanı ve Testler,3 Boyutlu Ekokardiyografi,Bilgisayar ve transdüser sistemlerindeki tekn...,2117
2,Tanı ve Testler,ACTH (Adrenokortikotropik Hormon) Testi,"ACTH (adrenokortikotropik hormon), hipofiz be...",2363
3,Tanı ve Testler,Akciğer Grafisi,"Akciğer grafisi; kalp, akciğerler, kan damarl...",1753
4,Tanı ve Testler,Albümin Testi,"Albümin testi, karaciğer ve böbrek fonksiyonl...",3426
...,...,...,...,...
136,Tanı ve Testler,Uyku Apnesi Testi,Kaliteli ve yeterli olmayan uyku süresi kişin...,3427
137,Tanı ve Testler,Üre (BUN Testi),"Üre, karaciğerin proteinin sindirimi sürecind...",3579
138,Tanı ve Testler,Üre Nefes Testi,Dünyanın en yaygın enfeksiyonlarından biri ol...,1142
139,Tanı ve Testler,Ürodinami Testi,Erkeklerde ve kadınlarda hemen hemen her yaşt...,2850


In [ ]:
df.to_pickle("hastaliklar.pkl")

# Tedavi Yöntemleri (Treatment Methods)

In [ ]:
param = "tedavi-yontemleri"
url = set_url(base_url, param)
url

'https://www.memorial.com.tr/tedavi-yontemleri'

In [ ]:
htmlData = get_divs(url)

In [ ]:
names = get_names(htmlData)
names[:3], len(names)

(['Ablasyon', 'Açık Kalp Ameliyatı', 'Açık Nefrektomi'], 207)

In [ ]:
name_params = []
for html in htmlData:
  url = [a['href'] for a in html.select('a[href^="/tedavi-yontemleri"]')]
  name_params.append(url[0])
len(name_params)

207

In [ ]:
# creating a list of dicts to keep track of the data while running ThreadPoolExecutor
param_records = []
for i in range(len(names)):
  rec = {
      "name": names[i],
      "param": name_params[i]
  }
  param_records.append(rec)

In [ ]:
raw_texts = []
get_raw_text_pool(10, param_records)      # takes about 30sec to complete

In [ ]:
# getting raw text and names in a list format
names = []
raws = []
for rec in raw_texts:
  names.append(list(rec.keys())[0])
  raws.append(list(rec.values())[0])

In [ ]:
texts = get_texts(raws, k=4)    # k was checked beforehand

In [ ]:
encoder = "cl100k_base"
num_tokens = get_num_tokens(encoder, texts)

In [ ]:
data = {
    "category": "Tedavi Yöntemleri",
    "topic": names,
    "text": texts,
    "num_tokens": num_tokens
}

In [ ]:
df = pd.DataFrame(data=data, index=None)

In [ ]:
df

,category,topic,text,num_tokens
0,Tedavi Yöntemleri,Açık Kalp Ameliyatı,Kalp ameliyatlarında en klasik yöntem olan aç...,1552
1,Tedavi Yöntemleri,Aferez,"Aferez, kişinin vücudundan kanının alınarak b...",1608
2,Tedavi Yöntemleri,Akciğer Kanseri,Tüm dünyada kanser türleri arasında en sık ha...,2206
3,Tedavi Yöntemleri,Ablasyon,Ablasyon anormal kalp ritmini düzeltmek amaçl...,2074
4,Tedavi Yöntemleri,Akıllı Lens,"Akıllı lens, göz içine yerleştirilen ve görme...",1445
...,...,...,...,...
202,Tedavi Yöntemleri,Yumurta Çatlatma İğnesi,"Yumurtalama sorunu yaşayıp, adet düzensizliği...",2008
203,Tedavi Yöntemleri,Yüksek Tibial Osteotomi Ameliyatı,"Diz eklemi, vücudun ağırlığını taşıyan karmaş...",1766
204,Tedavi Yöntemleri,Yüz Dolgusu,"Yüz dolgusu, çene, göz, elmacık kemikleri ve ...",2569
205,Tedavi Yöntemleri,Vitrektomi,"Vitrektomi, göz içerisinde yer alan vitreus s...",2377


In [ ]:
df.to_pickle("tedavi-yontemleri.pkl")

# Merging DataFrames

In [ ]:
df_1 = pd.read_pickle("tani-ve-testler.pkl")
df_2 = pd.read_pickle("hastaliklar.pkl")
df_3 = pd.read_pickle("tedavi-yontemleri.pkl")

In [ ]:
frames = [df_1, df_2, df_3]
result = pd.concat(frames, ignore_index=True)

In [ ]:
result

,category,topic,text,num_tokens
0,Tanı ve Testler,Tiroid Biyopsisi,Tiroid bezini oluşturan hücrelerin düzensiz ç...,3501
1,Tanı ve Testler,3 Boyutlu Ekokardiyografi,Bilgisayar ve transdüser sistemlerindeki tekn...,2117
2,Tanı ve Testler,ACTH (Adrenokortikotropik Hormon) Testi,"ACTH (adrenokortikotropik hormon), hipofiz be...",2363
3,Tanı ve Testler,Akciğer Grafisi,"Akciğer grafisi; kalp, akciğerler, kan damarl...",1753
4,Tanı ve Testler,Albümin Testi,"Albümin testi, karaciğer ve böbrek fonksiyonl...",3426
...,...,...,...,...
912,Tedavi Yöntemleri,Yumurta Çatlatma İğnesi,"Yumurtalama sorunu yaşayıp, adet düzensizliği...",2008
913,Tedavi Yöntemleri,Yüksek Tibial Osteotomi Ameliyatı,"Diz eklemi, vücudun ağırlığını taşıyan karmaş...",1766
914,Tedavi Yöntemleri,Yüz Dolgusu,"Yüz dolgusu, çene, göz, elmacık kemikleri ve ...",2569
915,Tedavi Yöntemleri,Vitrektomi,"Vitrektomi, göz içerisinde yer alan vitreus s...",2377


In [ ]:
result["num_tokens"].sum()    # total number of tokens

5227389

In [ ]:
result.to_pickle("MedData_P1.pkl")